In [1]:
import os
import time
import typing
import torch
import cv2
import numpy as np
import onnxruntime as ort
from collections import deque
from modelArc import CRNN
from itertools import groupby

In [ ]:

from mltu.inferenceModel import OnnxInferenceModel
from mltu.utils.text_utils import ctc_decoder, get_cer

model_path = r"D:\Projects\reciept-scanner\RCNN\models\202407310100\model.onnx"
val_path = r"D:\Projects\reciept-scanner\RCNN\models\202407310100\val.csv"

class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, (128, 32))

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(self.output_names, {self.input_names[0]: image_pred})[0]

        text = ctc_decoder(preds, self.metadata["vocab"])[0]

        return text

if __name__ == "__main__":
    import pandas as pd
    from tqdm import tqdm

    model = ImageToWordModel(model_path=model_path)

    df = pd.read_csv(val_path).values.tolist()

    accum_cer = []
    for image_path, label in tqdm(df):
        image = cv2.imread(image_path.replace("\\", "/"))

        prediction_text = model.predict(image)

        cer = get_cer(prediction_text, label)
        print(f"Image: {image_path}, Label: {label}, Prediction: {prediction_text}, CER: {cer}")

        accum_cer.append(cer)

    print(f"Average CER: {np.average(accum_cer)}")


In [7]:
model_path = r"D:\Projects\reciept-scanner\RCNN\models\202407310100\model.onnx"
img_path = r"D:\photos\RCNN4\BBOXES\69.jpg"
#"D:\photos\SORIE\test\0afc0804-7e6e-4e77-ab68-8d9384918be7.jpg"

class inferencemode:
    def __init__(self, model_path: str = ""):
        self.model_path = model_path.replace("\\", "/")
        providers = ["CUDAExecutionProvider", "CPUExecutionProvider"] if ort.get_device() == "GPU" else ["CPUExecutionProvider"]

        self.model = ort.InferenceSession(model_path, providers = providers)

        self.metadata = {}
        for key, value in self.model.get_modelmeta().custom_metadata_map.items():
            new_value = value
            self.metadata[key] = new_value

        self.input_shapes = [meta.shape for meta in self.model.get_inputs()]
        self.input_names = [meta.name for meta in self.model._inputs_meta]
        self.output_names = [meta.name for meta in self.model._outputs_meta]

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, (128, 32))

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(self.output_names, {self.input_names[0]: image_pred})[0]

        #highest prob
        argmax_preds = np.argmax(preds, axis=-1)
        grouped_preds = [[k for k,_ in groupby(preds)] for preds in argmax_preds]
        texts = ["".join([self.metadata["vocab"][k] for k in group if k < len(self.metadata["vocab"])]) for group in grouped_preds]
        text = texts[0]

        return text



model = inferencemode(model_path)

image = cv2.imread(img_path.replace("\\", "/"))

prediction_text = model.predict(image)

print(f"Prediction is: {prediction_text}, Image Path: {img_path}")


Prediction is: T0 26, Image Path: D:\photos\RCNN4\BBOXES\69.jpg
